In [14]:
import os
import pandas as pd
import json

import openai
import tiktoken

# Import and pass api key
with open(r'C:\Users\mhuh22\Desktop\openai_api.txt', 'r') as file:
    api_key = file.readline().strip()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [11]:
qa_df = pd.read_csv(r'UPDATED-MARCH-2023-Notebooks-and-Files\UPDATED-MARCH-2023-Notebooks-and-Files\08-Chat-Bot-with-Fine-Tuning\python_qa.csv')
qa_df.head(3)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,ParentId,Answer
0,11060,912.0,2008-08-14T13:59:21Z,NaN,18,How should I unit test a code-generator?,This is a difficult and open-ended question I ...,11060,I started writing up a summary of my experienc...
1,17250,394.0,2008-08-20T00:16:40Z,NaN,24,Create an encrypted ZIP file in Python,I'm creating an ZIP file with ZipFile in Pytho...,17250,I created a simple library to create a passwor...
2,31340,242853.0,2008-08-27T23:44:47Z,NaN,71,"How do threads work in Python, and what are co...",I've been trying to wrap my head around how th...,31340,"Yes, because of the Global Interpreter Lock (G..."


In [6]:
# Create lists from the two columns
questions,answers = qa_df['Body'], qa_df['Answer']

# Create a dictionary from the two lists
qa_openai_format = [ {'prompt':q, 'completion':a}  for q,a in zip(questions,answers)]

In [15]:
qa_openai_format[4]['prompt']

'I am taking a class that requires Python. We will review the language in class next week, and I am a quick study on new languages, but I was wondering if there are any really great Python books I can grab while I am struggling through the basics of setting up my IDE, server environment and all those other "gotchas" that come with a new programming language. Suggestions?\n'

In [21]:
response = openai.Completion.create(
    model = 'text-babbage-001',
    prompt = qa_openai_format[4]['prompt'],
    max_tokens = 250,
    temperature = 0
)

In [22]:
response['choices'][0]['text']

'\nThere are a few great Python books that you could consider while you are learning Python. One book that is particularly helpful is "Python for Data Science" by Geoffrey Hinton. This book is packed with information on data science and Python, and it is a great resource for anyone who wants to learn Python for data science purposes. Another great book to consider is "Python for Data Science Mastery" by Michael Nielsen. This book is designed to help you learn more about data science and Python, and it is a great resource for anyone who wants to learn more about Python for data science purposes.'

In [25]:
# Calculate the number of tokens from the string for cost purposes
def num_tokens_from_string(string,encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [39]:
dataset_size = 500

with open('example_training_data.json','w') as f:
    for entry in qa_openai_format:
        f.write(json.dumps(entry))
        f.write('\n')

In [43]:
token_counter = 0

for element in qa_openai_format[:500]:
    for prompt, completion in element.items():
        token_counter += num_tokens_from_string(prompt,'gpt2')
        token_counter += num_tokens_from_string(completion,'gpt2')
        
token_counter

197362

In [44]:
# GPT-3 model training cost
# $0.0006 / 1000 tokens (training) * 4 epochs
def compute_gpt_cost(tokens, version='gpt2'):
    if version == 'gpt2':
        cost = 0.0006 * 4 * token_counter / 1000
    return cost
        
compute_gpt_cost(token_counter)

0.4736688

In [41]:
len(qa_openai_format)

4429